# Exemplo 4: Vendas - Detecção de Fraude em Tempo Real (Spark ML)

Este notebook demonstra o uso de **Machine Learning** com **Spark Streaming** para identificar fraudes em transações de vendas.

**Cenário**: Treinar um modelo K-Means para agrupar transações normais e detectar anomalias (fraudes) em tempo real com base no valor da transação e distância geográfica.

## 1. Configuração do Ambiente

In [ ]:
# Instalar Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar e Instalar Spark
!wget https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz && tar xf spark-3.5.0-bin-hadoop3.tgz

# Baixar e Instalar Kafka
!wget https://archive.apache.org/dist/kafka/3.6.1/kafka_2.13-3.6.1.tgz && tar xf kafka_2.13-3.6.1.tgz

# Instalar pacotes Python
!pip install -q findspark pyspark kafka-python

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
import findspark
findspark.init()

## 2. Iniciar Kafka

In [ ]:
%%bash
cd kafka_2.13-3.6.1
bin/zookeeper-server-start.sh -daemon config/zookeeper.properties
sleep 5
bin/kafka-server-start.sh -daemon config/server.properties
sleep 5
bin/kafka-topics.sh --create --topic transactions --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1

## 3. Treinamento do Modelo (K-Means)
Simulamos um histórico de dados 'normais' para treinar o modelo.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
import random

spark = SparkSession.builder.appName("FraudDetection").getOrCreate()

# Gerar dados históricos (Normais: valor baixo/médio, distância baixa)
# Valores: 10-500, Distância: 0-50
data = []
for _ in range(1000):
    data.append((float(random.randint(10, 500)), float(random.randint(0, 50))))

# Adicionar algumas anomalias para o 'treino' não ficar viciado demais, ou deixar puramente normal
# Na verdade, K-Means vai achar o centroide dos dados normais.

df_train = spark.createDataFrame(data, ["amount", "distance"])

assembler = VectorAssembler(inputCols=["amount", "distance"], outputCol="features")
df_train_vec = assembler.transform(df_train)

# Treinar KMeans com k=2 (Pequenas/Normais vs Grandes/Distantes se houver, ou forçar apenas 1 cluster normal)
# Vamos supor k=1 para simplificar 'o que é normal' ou k=5 clusters de comportamento padrão.
kmeans = KMeans(k=2, seed=1)
model = kmeans.fit(df_train_vec)
centers = model.clusterCenters()
print("Centros dos Clusters:", centers)

## 4. Produtor de Transações em Tempo Real
Gera transações misturando legítimas e fraudes (valor altíssimo ou muito longe).

In [ ]:
import json
import time
from kafka import KafkaProducer
import threading

def generate_transactions():
    producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                             value_serializer=lambda x: json.dumps(x).encode('utf-8'))
    try:
        for _ in range(100):
            if random.random() > 0.9: # 10% de chance de fraude
                txn = {"amount": float(random.randint(1000, 5000)), "distance": float(random.randint(100, 500))}
            else:
                txn = {"amount": float(random.randint(10, 500)), "distance": float(random.randint(0, 50))}
            
            producer.send('transactions', value=txn)
            time.sleep(0.5)
    finally:
        producer.close()

thread = threading.Thread(target=generate_transactions)
thread.start()

## 5. Detecção em Tempo Real
Consome do Kafka e aplica o modelo.

In [ ]:
%%writefile kafka_consumer.py
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
import random

spark = SparkSession.builder.appName("FraudDetection").getOrCreate()

# Gerar dados históricos (Normais: valor baixo/médio, distância baixa)
# Valores: 10-500, Distância: 0-50
data = []
for _ in range(1000):
    data.append((float(random.randint(10, 500)), float(random.randint(0, 50))))

# Adicionar algumas anomalias para o 'treino' não ficar viciado demais, ou deixar puramente normal
# Na verdade, K-Means vai achar o centroide dos dados normais.

df_train = spark.createDataFrame(data, ["amount", "distance"])

assembler = VectorAssembler(inputCols=["amount", "distance"], outputCol="features")
df_train_vec = assembler.transform(df_train)

# Treinar KMeans com k=2 (Pequenas/Normais vs Grandes/Distantes se houver, ou forçar apenas 1 cluster normal)
# Vamos supor k=1 para simplificar 'o que é normal' ou k=5 clusters de comportamento padrão.
kmeans = KMeans(k=2, seed=1)
model = kmeans.fit(df_train_vec)
centers = model.clusterCenters()
print("Centros dos Clusters:", centers)
# --- Streaming Logic ---
from pyspark.sql.functions import from_json, col, udf
from pyspark.sql.types import DoubleType

schema = StructType([
    StructField("amount", DoubleType()),
    StructField("distance", DoubleType())
])

df_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "transactions") \
    .load()

txns = df_stream.select(from_json(col("value").cast("string"), schema).alias("data")).select("data.*")

# Obs: Streaming MLPipelins são complexos. Uma abordagem simples é usar 'foreachBatch' ou aplicar o modelo transformado se suportado.
# O K-MeansModel do Spark suporta transform em streaming?
# Na versão 3.5, sim, 'transform' pode ser usado.

# Precisa remontar o VectorAssembler nas features de streaming
# Nota: Para usar ML Transformers em streaming, é preciso que eles sejam stateless ou o modelo já esteja fitado.

assembler_stream = VectorAssembler(inputCols=["amount", "distance"], outputCol="features")
txns_vec = assembler_stream.transform(txns)

predictions = model.transform(txns_vec)

# Vamos considerar 'fraude' se a predição cair em um cluster específico OU se tivermos lógica de distância.
# Para este exemplo simples, vamos apenas exibir o cluster predito.
# Se tivéssemos definido que cluster 1 são 'valores altos', filtraríamos prediction == 1.

query = predictions.select("amount", "distance", "prediction") \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()


query.awaitTermination()

In [ ]:
!spark-submit --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1 kafka_consumer.py